In [ ]:
# m4a -> mp3

!pip install pydub
import os
from pydub import AudioSegment

# 폴더 경로 설정
folder_path = '/content/drive/MyDrive/PBL/ai_voice/human/hs'

# 폴더 내의 모든 파일에 대해
for filename in os.listdir(folder_path):
    # 만약 확장자가 .m4a인 파일이면
    if filename.endswith('.m4a'):
        m4a_path = os.path.join(folder_path, filename)
        # MP3 파일 이름 생성 (기존 확장자 .m4a를 .mp3로 바꿈)
        mp3_path = os.path.join(folder_path, f'{os.path.splitext(filename)[0]}.mp3')

        # M4A 파일을 MP3로 변환
        audio = AudioSegment.from_file(m4a_path, format="m4a")
        audio.export(mp3_path, format="mp3")

        # 원본 M4A 파일 삭제
        os.remove(m4a_path)

print('모든 m4a파일을 mp3로 변환하고 원본 m4a파일 삭제 완료')

In [ ]:
# wav -> mp3

from pydub import AudioSegment

# 파일 경로 설정
wav_path = '/content/drive/MyDrive/PBL/ai_voice/ai/vocals.wav'

# MP3 파일 이름 생성 (기존 확장자 .wav를 .mp3로 바꿈)
mp3_path = '/content/drive/MyDrive/PBL/ai_voice/ai/vocals.mp3'

# WAV 파일을 MP3로 변환
audio = AudioSegment.from_file(wav_path, format="wav")
audio.export(mp3_path, format="mp3")

# 원본 WAV 파일 삭제
os.remove(wav_path)

print('WAV파일을 MP3로 변환하고 원본 WAV파일 삭제 완료')


In [ ]:
# mp3 부분 추출

from pydub import AudioSegment

# MP3 파일 로드
audio = AudioSegment.from_mp3("/content/drive/MyDrive/PBL/ai_voice/ai/vocals.mp3")

# 27초부터 37초까지의 부분 추출 (PyDub은 밀리세컨드 단위를 사용합니다)
extracted = audio[27000:37000]

# 추출한 부분을 새로운 MP3 파일로 저장
extracted.export("/content/drive/MyDrive/PBL/ai_voice/ai/vocals_10sec.mp3", format="mp3")


In [ ]:
#정확도 확인을 새나 목소리로 확인

# Gmail 폴더의 음성파일들
gmail_audio_files = get_files_from_dir('/content/drive/MyDrive/PBL/ai_voice/human/Gmail', 'mp3')

# 특징 추출
gmail_features = [extract_features(audio_file,max_length) for audio_file in gmail_audio_files]

# 모델에 입력할 수 있는 형태로 변환
X_gmail = np.vstack(gmail_features)
X_gmail = X_gmail.reshape(X_gmail.shape[0], X_gmail.shape[1], 1)

# 예측 수행 (모델의 출력은 각 클래스에 대한 확률 분포이므로 argmax 함수를 사용하여 가장 확률이 높은 클래스 선택)
y_pred = np.argmax(model.predict(X_gmail), axis=1)

# 정답 라벨 생성 (Gmail 폴더의 모든 파일은 human이므로 1로 채워진 배열 생성)
y_true = np.ones(len(gmail_audio_files))

# 정확도 계산 및 출력
accuracy = np.sum(y_true == y_pred) / len(y_true)
print(f'Accuracy: {accuracy * 100:.2f}%')


In [ ]:
#노래추출 정확도 확인

# 음성 파일 경로
audio_file = '/content/drive/MyDrive/PBL/ai_voice/ai/music.mp3'

# 특징 추출
features = extract_features(audio_file, max_length)

# 모델에 입력할 수 있는 형태로 변환
X = features.reshape(1, features.shape[0], 1)

# 예측 수행 (모델의 출력은 각 클래스에 대한 확률 분포이므로 argmax 함수를 사용하여 가장 확률이 높은 클래스 선택)
y_pred = np.argmax(model.predict(X), axis=1)

# 정답 라벨 생성 (음성 파일이 AI 목소리인 경우 0으로 설정)
y_true = np.zeros(1)

# 정확도 계산 및 출력
accuracy = np.sum(y_true == y_pred) / len(y_true)
print(f'Accuracy: {accuracy * 100:.2f}%')


In [ ]:
#mp3 -> wav, fft

import numpy as np
from pydub import AudioSegment
from IPython.display import Audio
from scipy.io.wavfile import write, read

# MP3 파일 로드 및 재생
audio = AudioSegment.from_mp3("/content/drive/MyDrive/PBL/ai_voice/ai/ai_phone_call_noise.mp3")
Audio(audio.raw_data, rate=audio.frame_rate)

# MP3 파일을 WAV로 변환 (scipy에서 읽기 위함)
audio.export("/tmp/audio.wav", format="wav")

# WAV 파일 로드 및 FFT 적용
rate, data = read('/tmp/audio.wav')
fft_result = np.fft.rfft(data)

# 주파수 범위 설정 (100~250Hz)
low_freq_boundary = int(500 * len(data) / rate)
high_freq_boundary = int(2500 * len(data) / rate)

# 필터 적용: 설정된 범위 외의 주파수 성분 제거
fft_result[:low_freq_boundary] = 0
fft_result[high_freq_boundary:] = 0

# 역 FFT 적용 및 오디오 데이터 복원
cleaned_data = np.fft.irfft(fft_result)

# 결과 저장 및 재생
write("/content/drive/MyDrive/PBL/ai_voice/ai/ai_phone_call_noise_cancel.wav", rate, cleaned_data.astype(np.int16))
AudioSegment.from_wav("/content/drive/MyDrive/PBL/ai_voice/ai/ai_phone_call_noise_cancel.wav")

